In [11]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import matplotlib.dates as mdates
from matplotlib import rcParams
import datetime
import os
from datetime import timedelta
from datetime import date
from dateutil.relativedelta import relativedelta
import numpy as np
import fnmatch
from matplotlib.backends.backend_pdf import PdfPages
import plotly.graph_objects as go
from dash import Dash, html, dcc, callback, Output, Input, dash_table, State
import plotly.express as px
from statistics import mean

In [12]:
#1

# Cleaning Data

path = 'C:/Users/mrcoo/Box/TREX/MISCELLANEOUS/Datalogger_Report_Files'
sites = ['OLA', 'WWF', 'VAC', 'SLC', 'FLT', 'WES']

all_st = []

for i in range(0,6):
  appended_data=[]
  for fname in fnmatch.filter(os.listdir(path),sites[i]+'*CSIFormat*'):
    df = pd.read_csv(path+'/'+fname, header=[0],skiprows=[0,2,3],sep=',',na_values="NAN",engine='python')
    df=df.reset_index(drop=True)
    df.TIMESTAMP= pd.to_datetime(df['TIMESTAMP'], format= 'mixed')
    appended_data.append(df)
  dff=pd.concat(appended_data)
  dff=dff.sort_values(by=['TIMESTAMP'])
  dff=dff.drop_duplicates()
  #dff.to_csv('/Users/mina.swintek/Box/TREX/MISCELLANEOUS/Datalogger_Report_Files/output/' +sites[i]+ '_out.csv',index=False,na_rep='NAN')
  dff_all=dff
  all_st.append(dff_all)
df_all=pd.concat(all_st)
#df_all.to_csv('/Users/mina.swintek/Box/TREX/MISCELLANEOUS/Datalogger_Report_Files/output/All_stations_out.csv',index=False,na_rep='NAN')


df_all=df_all.reset_index(drop=True)
df_all=df_all.drop_duplicates()
#df_all.TIMESTAMP= pd.to_datetime(df_all['TIMESTAMP'], format= '%Y-%m-%d %H:%M:%S')
start_date = pd.Timestamp('2023-08-01')
mask = (df_all['TIMESTAMP'] >= start_date)
df_all = df_all.loc[mask]
df_all['ET'] = np.where(df_all['ET']<=0 , np.nan, df_all['ET'])
df_all['FC_mass'] = np.where(df_all['FC_mass']<-2 , np.nan, df_all['FC_mass'])
df_all['FC_mass'] = np.where(df_all['FC_mass']>1 , np.nan, df_all['FC_mass'])
df_all['TS5_2cm'] = np.where(df_all['TS5_2cm']<-10 , np.nan, df_all['TS5_2cm'])
df_all['TS5_2cm'] = np.where(df_all['TS5_2cm']> 100 , np.nan, df_all['TS5_2cm'])
VAC1 = df_all[df_all.Site=='VAC']
WWF1 = df_all[df_all.Site=='WWF']
OLA1 = df_all[df_all.Site=='OLA']
SLC1 = df_all[df_all.Site=='SLC']
FLT1 = df_all[df_all.Site=='FLT']
WES1 = df_all[df_all.Site=='WES']


rangepath = r'\Users\mrcoo\Box\TREX\MISCELLANEOUS\Datalogger_Report_Files\suplementary\all_dl_ranges.csv'
rangedf = pd.read_csv(rangepath, header=[0],sep=',',na_values="NAN",engine='python')

rangedf.at[0, 'e_probe'] = -2
rangedf.at[0, 'H2O_probe'] = -10
rangedf.at[0, 'RH_3_1_1'] = -100
rangedf.at[0, 'T_DP_3_1_1'] = -100
rangedf.at[0, 'P_Tot'] = -2

all_calls = ['batt_volt', 'SG_1_1_1', 'SG_2_1_1','SG_3_1_1','SG_4_1_1','SG_5_1_1', 'TS1_2cm', 'TS1_6cm', 'TS2_2cm','TS2_6cm', 'TS3_2cm','TS3_6cm', 'TS4_2cm', 'TS4_6cm', 'TS5_2cm', 'TS5_6cm', 'G_plate_1_1_1', 'G_plate_2_1_1', 'G_plate_3_1_1', 'G_plate_4_1_1', 'G_plate_5_1_1', 'G', 'G_1_1_1',
              'hydra1_temp', 'SoilWater_1', 'G_2_1_1', 'hydra2_temp', 'SoilWater_2', 'G_3_1_1', 'hydra3_temp', 'SoilWater_3', 'G_4_1_1', 'hydra4_temp', 'SoilWater_4', 'G_5_1_1', 'hydra5_temp', 'SoilWater_5', 'FW', 'FW_SIGMA', 'H_FW', 'LW_IN', 'LW_OUT', 'NETRAD', 'SW_IN', 'SW_OUT',
             'H', 'T_SONIC', 'T_SONIC_SIGMA', 'TAU', 'TAU_QC', 'TKE', 'TSTAR', 'USTAR', 'Ux', 'Ux_SIGMA', 'Uy', 'Uy_SIGMA', 'Uz', 'Uz_SIGMA', 'WD', 'WD_SIGMA', 'WD_SONIC', 'WS', 'WS_MAX', 'WS_RSLT', 'Bowen_ratio', 'ET', 'LE', 'energy_closure', 'CO2_density',
             'H2O_density', 'CO2_density_SIGMA', 'H2O_density_SIGMA', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min', 'RH_2_1_1', 'T_DP_2_1_1', 'TA_2_1_1', 'FC_mass', 'e', 'e_sat', 'amb_e', 'amb_e_sat',
             'PA', 'RH_1_1_1', 'T_DP_1_1_1', 'TA_1_1_1', 'Duty_cycle_TS100_Fan_Avg', 'e_probe', 'e_sat_probe', 'Freq_tach_TS100_Avg', 'H2O_probe', 'RH_3_1_1', 'T_DP_3_1_1', 'TA_3_1_1', 'VPD', 'T_CANOPY', 'T_SI111_body', 'air_mass_coeff','FETCH_MAX', 'FP_DIST_INTRST', 'hour_angle', 'sun_azimuth', 'sun_declination', 'sun_elevation']


test_calls = ['FW', 'H_FW', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'TA_3_1_1', 'T_CANOPY', 'G', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min', 'CO2_density', 'H2O_density', 'LE', 'H', 'VPD', 'P_Tot', 'batt_volt']



In [23]:
#2 TREX

path = 'C:/Users/mrcoo/Box/TREX/MISCELLANEOUS/Datalogger_Report_Files'
sites = ['OLA', 'WWF', 'VAC', 'SLC', 'FLT', 'WES']

all_st = []

for i in range(0,6):
  appended_data=[]
  for fname in fnmatch.filter(os.listdir(path),sites[i]+'*CSIFormat*'):
    print(fname)
    df = pd.read_csv(path+'/'+fname, header=[0],skiprows=[0,2,3],sep=',',na_values="NAN",engine='python')
    df=df.reset_index(drop=True)
    df.TIMESTAMP= pd.to_datetime(df['TIMESTAMP'], format= 'mixed')
    appended_data.append(df)
  dff=pd.concat(appended_data)
  dff=dff.sort_values(by=['TIMESTAMP'])
  dff=dff.drop_duplicates()
  #dff.to_csv('/Users/mina.swintek/Box/TREX/MISCELLANEOUS/Datalogger_Report_Files/output/' +sites[i]+ '_out.csv',index=False,na_rep='NAN')
  dff_all=dff
  all_st.append(dff_all)
df_all=pd.concat(all_st)
#df_all.to_csv('/Users/mina.swintek/Box/TREX/MISCELLANEOUS/Datalogger_Report_Files/output/All_stations_out.csv',index=False,na_rep='NAN')


df_all=df_all.reset_index(drop=True)
df_all=df_all.drop_duplicates()
#df_all.TIMESTAMP= pd.to_datetime(df_all['TIMESTAMP'], format= '%Y-%m-%d %H:%M:%S')
start_date = pd.Timestamp('2023-08-01')
mask = (df_all['TIMESTAMP'] >= start_date)
df_all = df_all.loc[mask]
df_all['ET'] = np.where(df_all['ET']<=0 , np.nan, df_all['ET'])
df_all['FC_mass'] = np.where(df_all['FC_mass']<-2 , np.nan, df_all['FC_mass'])
df_all['FC_mass'] = np.where(df_all['FC_mass']>1 , np.nan, df_all['FC_mass'])
df_all['TS5_2cm'] = np.where(df_all['TS5_2cm']<-10 , np.nan, df_all['TS5_2cm'])
df_all['TS5_2cm'] = np.where(df_all['TS5_2cm']> 100 , np.nan, df_all['TS5_2cm'])
# VAC2 = df_all[df_all.Site=='VAC']
# WWF2 = df_all[df_all.Site=='WWF']
# OLA2 = df_all[df_all.Site=='OLA']
# SLC2 = df_all[df_all.Site=='SLC']
# FLT2 = df_all[df_all.Site=='FLT']
# WES2 = df_all[df_all.Site=='WES']


# VACS = [VAC1, VAC2]
# VAC = pd.concat(VACS)
# VAC = VAC.sort_values(by=['TIMESTAMP'])
# VAC = VAC.drop_duplicates()

# WWFS = [WWF1, WWF2]
# WWF = pd.concat(WWFS)
# WWF = WWF.sort_values(by=['TIMESTAMP'])
# WWF = WWF.drop_duplicates()

# OLAS = [OLA1, OLA2]
# OLA = pd.concat(OLAS)
# OLA = OLA.sort_values(by=['TIMESTAMP'])
# OLA = OLA.drop_duplicates()

# SLCS = [SLC1, SLC2]
# SLC = pd.concat(SLCS)
# SLC = SLC.sort_values(by=['TIMESTAMP'])
# SLC = SLC.drop_duplicates()

# FLTS = [FLT1, FLT2]
# FLT = pd.concat(FLTS)
# FLT = FLT.sort_values(by=['TIMESTAMP'])
# FLT = FLT.drop_duplicates()

# WESS = [WES1, WES2]
# WES = pd.concat(WESS)
# WES = WES.sort_values(by=['TIMESTAMP'])
# WES = WES.drop_duplicates()

trex_all = df_all
# trex_all.to_csv("C:/Users/mrcoo/Desktop/Audrey Work/Dashboard/Data/trex_data.csv")

OLA_Flux_CSIFormat.dat
WWF_Flux_CSIFormat.dat
VAC_Flux_CSIFormat.dat
SLC_Flux_CSIFormat.dat
FLT_Flux_CSIFormat.dat
WES_Flux_CSIFormat.dat


In [14]:
test_calls = ['e_probe', 'e_sat_probe', 'H2O_probe', 'RH_3_1_1', 'T_DP_3_1_1', 'FW', 'H_FW', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'TA_3_1_1', 'T_CANOPY', 'G', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min', 'CO2_density', 'H2O_density', 'LE', 'H', 'VPD', 'P_Tot', 'batt_volt']



In [15]:
#1 matt

path = path = 'C:/Users/mrcoo/Box/TREX/MISCELLANEOUS/Datalogger_Report_Files/suplementary/old_datafiles/Datalogger_Data_May_2024'
sites = ['BLS_001', 'BLS_002', 'ORO_022', 'ORO_043', 'COR_CS3', 'ART_011', "BRO_001"]

all_st = []

for i in range(0,6):
  appended_data=[]
  for fname in fnmatch.filter(os.listdir(path),sites[i]+'*CSIFormat1*'):
    df = pd.read_csv(path+'/'+fname, header=[0],skiprows=[0,2,3],sep=',',na_values="NAN",engine='python')
    df=df.reset_index(drop=True)
    df.TIMESTAMP= pd.to_datetime(df['TIMESTAMP'], format= 'mixed')
    appended_data.append(df)
  dff=pd.concat(appended_data)
  dff=dff.sort_values(by=['TIMESTAMP'])
  dff=dff.drop_duplicates()
  dff['Site'] = sites[i]
  #dff.to_csv('/Users/mina.swintek/Box/TREX/MISCELLANEOUS/Datalogger_Report_Files/output/'+sites[i]+'_out.csv',index=False,na_rep='NAN')

  dff_all=dff
  all_st.append(dff_all)
df_all=pd.concat(all_st)
#df_all.to_csv('/Users/mina.swintek/Box/TREX/MISCELLANEOUS/Datalogger_Report_Files/output/Matt_All_stations_out.csv',index=False,na_rep='NAN')

df_all=df_all.reset_index(drop=True)
df_all=df_all.drop_duplicates()

#df_all['ET'] = np.where(df_all['ET']<=0 , np.nan, df_all['ET'])
df_all['FC_mass'] = np.where(df_all['FC_mass']<-2 , np.nan, df_all['FC_mass'])
df_all['FC_mass'] = np.where(df_all['FC_mass']>1 , np.nan, df_all['FC_mass'])
df_all['TS5_2cm'] = np.where(df_all['TS5_2cm']<-10 , np.nan, df_all['TS5_2cm'])
df_all['TS5_2cm'] = np.where(df_all['TS5_2cm']> 100 , np.nan, df_all['TS5_2cm'])
BL1 = df_all[df_all.Site=='BLS_001']
BL2 = df_all[df_all.Site=='BLS_002']
OR2 = df_all[df_all.Site=='ORO_022']
OR4 = df_all[df_all.Site=='ORO_043']
COR = df_all[df_all.Site=='COR_CS3']
ART = df_all[df_all.Site=='ART_011']


rangepath = r'\Users\mrcoo\Box\TREX\MISCELLANEOUS\Datalogger_Report_Files\suplementary\all_dl_ranges.csv'
rangedf = pd.read_csv(rangepath, header=[0],sep=',',na_values="NAN",engine='python')

all = ['TIMESTAMP', 'RECORD', 'V_batt', 'FC_mass', 'FC_QC', 'FC_samples', 'LE', 'LE_QC', 'LE_samples', 'H', 'H_QC', 'H_samples', 'NETRAD', 'G', 'G_1_1_1', 'G_2_1_1', 'G_3_1_1', 'G_4_1_1', 'G_5_1_1', 'SG_1_1_1',
       'SG_2_1_1', 'SG_3_1_1', 'SG_4_1_1', 'SG_5_1_1', 'G_plate_1_1_1', 'G_plate_2_1_1', 'G_plate_3_1_1', 'G_plate_4_1_1', 'G_plate_5_1_1', 'energy_closure', 'Bowen_ratio', 'TAU', 'TAU_QC', 'USTAR', 'TSTAR',
       'TKE', 'TA_1_1_1', 'RH_1_1_1', 'T_DP_1_1_1', 'e_amb', 'e_sat_amb', 'TA_1_1_2', 'RH_1_1_2', 'T_DP_1_1_2', 'e', 'e_sat', 'PA', 'VPD', 'Ux', 'Ux_SIGMA', 'Uy', 'Uy_SIGMA', 'Uz', 'Uz_SIGMA', 'T_SONIC',
       'T_SONIC_SIGMA', 'sonic_azimuth', 'WS', 'WS_RSLT', 'WD_SONIC', 'WD_SIGMA', 'WD', 'WS_MAX', 'CO2_density', 'CO2_density_SIGMA', 'H2O_density', 'H2O_density_SIGMA', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min',
       'ALB', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'sun_azimuth', 'sun_elevation', 'hour_angle', 'sun_declination', 'air_mass_coeff', 'daytime', 'TS1_2cm', 'TS1_6cm', 'TS2_2cm', 'TS2_6cm', 'TS3_2cm', 'TS3_6cm',
       'TS4_2cm', 'TS4_6cm', 'TS5_2cm', 'TS5_6cm', 'SWC_1_1_1', 'SWC_2_1_1', 'SWC_3_1_1', 'SWC_4_1_1', 'SWC_5_1_1', 'TS_CS65X_1_1_1', 'TS_CS65X_1_1_2', 'TS_CS65X_1_1_3', 'TS_CS65X_1_1_4', 'TS_CS65X_1_1_5',
       'cs65x_ec_1_1_1', 'cs65x_ec_1_1_2', 'cs65x_ec_1_1_3', 'cs65x_ec_1_1_4', 'cs65x_ec_1_1_5', 'FETCH_MAX', 'FETCH_90', 'FETCH_55', 'FETCH_40', 'UPWND_DIST_INTRST',
       'poor_enrg_clsur', 'TA_1_1_3', 'RH_1_1_3', 'T_DP_1_1_3', 'e_probe', 'e_sat_probe', 'H2O_density_probe', 'P','T_nr_in', 'T_nr_out', 'T_CANOPY', 'T_SI111_body', 'T_nr', 'R_LW_in_meas', 'R_LW_out_meas']


reports = ['SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'TA_1_1_3', 'T_CANOPY', 'G', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min', 'CO2_density', 'H2O_density', 'LE', 'H', 'VPD', 'P', 'V_batt']
issues = ['SWC_5_1_1', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'TA_1_1_3', 'T_CANOPY', 'G', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min', 'CO2_density', 'H2O_density', 'LE', 'H', 'VPD', 'P', 'V_batt']


In [16]:
winds = ['Ux', 'Ux_SIGMA', 'Uy', 'Uy_SIGMA', 'Uz', 'Uz_SIGMA', 'WS', 'WS_RSLT', 'WD_SONIC', 'WD_SIGMA', 'WD', 'WS_MAX']


thurs = ['G', 'G_1_1_1', 'G_2_1_1', 'G_3_1_1', 'G_4_1_1', 'G_5_1_1', 'SG_1_1_1', 'SG_2_1_1', 'SG_3_1_1', 'SG_4_1_1', 'SG_5_1_1', 
       'G_plate_1_1_1', 'G_plate_2_1_1', 'G_plate_3_1_1', 'G_plate_4_1_1', 'G_plate_5_1_1',
       'TS1_2cm', 'TS1_6cm', 'TS2_2cm', 'TS2_6cm', 'TS3_2cm', 'TS3_6cm', 'TS4_2cm', 'TS4_6cm', 'TS5_2cm', 'TS5_6cm', 
       'SWC_1_1_1', 'SWC_2_1_1', 'SWC_3_1_1', 'SWC_4_1_1', 'SWC_5_1_1', 
       'TS_CS65X_1_1_1', 'TS_CS65X_1_1_2', 'TS_CS65X_1_1_3', 'TS_CS65X_1_1_4', 'TS_CS65X_1_1_5', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'TA_1_1_3', 'T_CANOPY', 'G', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min', 'CO2_density', 'H2O_density', 'LE', 'H', 'VPD', 'P', 'V_batt']


rangedf.at[1, 'cs65x_ec_1_1_1'] = 2
rangedf.at[1, 'cs65x_ec_1_1_2'] = 2
rangedf.at[1, 'cs65x_ec_1_1_3'] = 2
rangedf.at[1, 'cs65x_ec_1_1_4'] = 2
rangedf.at[1, 'cs65x_ec_1_1_5'] = 2


In [17]:
test = ['FC_mass', 'FC_QC', 'FC_samples', 'LE', 'LE_QC', 'LE_samples', 'H', 'H_QC', 'H_samples', 'NETRAD', 'energy_closure', 'Bowen_ratio', 'TAU', 'TAU_QC', 'USTAR', 'TSTAR',
       'TKE', 'TA_1_1_1', 'RH_1_1_1', 'T_DP_1_1_1', 'e_amb', 'e_sat_amb', 'TA_1_1_2', 'RH_1_1_2', 'T_DP_1_1_2', 'e', 'e_sat', 'PA', 'VPD', 'Ux', 'Ux_SIGMA', 'Uy', 'Uy_SIGMA', 'Uz', 'Uz_SIGMA', 'T_SONIC',
       'T_SONIC_SIGMA', 'sonic_azimuth', 'WS', 'WS_RSLT', 'WD_SONIC', 'WD_SIGMA', 'WD', 'WS_MAX', 'CO2_density', 'CO2_density_SIGMA', 'H2O_density', 'H2O_density_SIGMA', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min',
       'ALB', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'sun_azimuth', 'sun_elevation', 'hour_angle', 'sun_declination', 'air_mass_coeff', 'daytime', 'FETCH_MAX', 'FETCH_90', 'FETCH_55', 'FETCH_40', 'UPWND_DIST_INTRST',
       'poor_enrg_clsur', 'TA_1_1_3', 'RH_1_1_3', 'T_DP_1_1_3', 'e_probe', 'e_sat_probe', 'H2O_density_probe', 'P','T_nr_in', 'T_nr_out', 'T_CANOPY', 'T_SI111_body', 'T_nr', 'R_LW_in_meas', 'R_LW_out_meas']

issues = ['SWC_5_1_1', 'FC_samples', 'LE_samples', 'H_samples', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'TA_1_1_3', 'T_CANOPY', 'G', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min', 'CO2_density', 'H2O_density', 'LE', 'H', 'VPD', 'P', 'V_batt']


In [22]:
#2 MATT

path = 'C:/Users/mrcoo/Box/TREX/MISCELLANEOUS/Datalogger_Report_Files'
sites = ['BLS_001', 'BLS_002', 'ORO_022', 'ORO_043', 'COR_CS3', 'ART_011', 'BRO_001']

all_st = []

for i in range(0,7):
  appended_data=[]
  for fname in fnmatch.filter(os.listdir(path),sites[i]+'*CSIFormat*'):
    df = pd.read_csv(path+'/'+fname, header=[0],skiprows=[0,2,3],sep=',',na_values="NAN",engine='python')
    df=df.reset_index(drop=True)
    appended_data.append(df)
  dff=pd.concat(appended_data)
  dff=dff.sort_values(by=['TIMESTAMP'])
  dff=dff.drop_duplicates()
  dff['Site'] = sites[i]
  dff_all=dff
  all_st.append(dff_all)

df_all=pd.concat(all_st)
df_all=df_all.reset_index(drop=True)
df_all=df_all.drop_duplicates()
df_all.TIMESTAMP= pd.to_datetime(df_all['TIMESTAMP'], format= '%Y-%m-%d %H:%M:%S')
df_all['FC_mass'] = np.where(df_all['FC_mass']<-2 , np.nan, df_all['FC_mass'])
df_all['FC_mass'] = np.where(df_all['FC_mass']>1 , np.nan, df_all['FC_mass'])
df_all['TS5_2cm'] = np.where(df_all['TS5_2cm']<-10 , np.nan, df_all['TS5_2cm'])
df_all['TS5_2cm'] = np.where(df_all['TS5_2cm']> 100 , np.nan, df_all['TS5_2cm'])

BL12 = df_all[df_all.Site=='BLS_001']
BL22 = df_all[df_all.Site=='BLS_002']
OR22 = df_all[df_all.Site=='ORO_022']
OR42 = df_all[df_all.Site=='ORO_043']
COR2 = df_all[df_all.Site=='COR_CS3']
ART2 = df_all[df_all.Site=='ART_011']
BRO = df_all[df_all.Site=='BRO_001']

BL1S = [BL1, BL12]
BL1 = pd.concat(BL1S)
BL1=BL1.sort_values(by=['TIMESTAMP'])
BL1=BL1.drop_duplicates()

BL2S = [BL2, BL22]
BL2 = pd.concat(BL2S)
BL2=BL2.sort_values(by=['TIMESTAMP'])
BL2=BL2.drop_duplicates()

OR2S = [OR2, OR22]
OR2 = pd.concat(OR2S)
OR2=OR2.sort_values(by=['TIMESTAMP'])
OR2=OR2.drop_duplicates()

OR4S = [OR4, OR42]
OR4 = pd.concat(OR4S)
OR4=OR4.sort_values(by=['TIMESTAMP'])
OR4=OR4.drop_duplicates()

CORS = [COR, COR2]
COR = pd.concat(CORS)
COR=COR.sort_values(by=['TIMESTAMP'])
COR=COR.drop_duplicates()

ARTS = [ART, ART2]
ART = pd.concat(ARTS)
ART=ART.sort_values(by=['TIMESTAMP'])
ART=ART.drop_duplicates()

BRO = BRO.sort_values(by=['TIMESTAMP'])
BRO=BRO.drop_duplicates()

all_ = ['V_batt', 'FC_mass', 'FC_QC', 'FC_samples', 'LE', 'LE_QC', 'LE_samples', 'H', 'H_QC', 'H_samples', 'NETRAD', 'G', 'G_1_1_1', 'G_2_1_1', 'G_3_1_1', 'G_4_1_1', 'G_5_1_1', 'SG_1_1_1',
       'SG_2_1_1', 'SG_3_1_1', 'SG_4_1_1', 'SG_5_1_1', 'G_plate_1_1_1', 'G_plate_2_1_1', 'G_plate_3_1_1', 'G_plate_4_1_1', 'G_plate_5_1_1', 'energy_closure', 'Bowen_ratio', 'TAU', 'TAU_QC', 'USTAR', 'TSTAR',
       'TKE', 'TA_1_1_1', 'RH_1_1_1', 'T_DP_1_1_1', 'e_amb', 'e_sat_amb', 'TA_1_1_2', 'RH_1_1_2', 'T_DP_1_1_2', 'e', 'e_sat', 'PA', 'VPD', 'Ux', 'Ux_SIGMA', 'Uy', 'Uy_SIGMA', 'Uz', 'Uz_SIGMA', 'T_SONIC',
       'T_SONIC_SIGMA', 'sonic_azimuth', 'WS', 'WS_RSLT', 'WD_SONIC', 'WD_SIGMA', 'WD', 'WS_MAX', 'CO2_density', 'CO2_density_SIGMA', 'H2O_density', 'H2O_density_SIGMA', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min',
       'ALB', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'sun_azimuth', 'sun_elevation', 'hour_angle', 'sun_declination', 'air_mass_coeff', 'daytime', 'TS1_2cm', 'TS1_6cm', 'TS2_2cm', 'TS2_6cm', 'TS3_2cm', 'TS3_6cm',
       'TS4_2cm', 'TS4_6cm', 'TS5_2cm', 'TS5_6cm', 'SWC_1_1_1', 'SWC_2_1_1', 'SWC_3_1_1', 'SWC_4_1_1', 'SWC_5_1_1', 'TS_CS65X_1_1_1', 'TS_CS65X_1_1_2', 'TS_CS65X_1_1_3', 'TS_CS65X_1_1_4', 'TS_CS65X_1_1_5',
       'cs65x_ec_1_1_1', 'cs65x_ec_1_1_2', 'cs65x_ec_1_1_3', 'cs65x_ec_1_1_4', 'cs65x_ec_1_1_5', 'FETCH_MAX', 'FETCH_90', 'FETCH_55', 'FETCH_40', 'UPWND_DIST_INTRST',
       'poor_enrg_clsur', 'TA_1_1_3', 'RH_1_1_3', 'T_DP_1_1_3', 'e_probe', 'e_sat_probe', 'H2O_density_probe', 'P','T_nr_in', 'T_nr_out', 'T_CANOPY', 'T_SI111_body', 'T_nr', 'R_LW_in_meas', 'R_LW_out_meas']


reports = ['SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'TA_1_1_3', 'T_CANOPY', 'G', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min', 'CO2_density', 'H2O_density', 'LE', 'H', 'VPD', 'P', 'V_batt']
issues = ['SWC_5_1_1', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'TA_1_1_3', 'T_CANOPY', 'G', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min', 'CO2_density', 'H2O_density', 'LE', 'H', 'VPD', 'P', 'V_batt']

matt_all = df_all

# matt_all.to_csv("C:/Users/mrcoo/Desktop/Audrey Work/Dashboard/Data/matt_data.csv")

In [19]:
reports = ['G', 'G_1_1_1', 'G_2_1_1', 'G_3_1_1', 'G_4_1_1', 'G_5_1_1', 'SG_1_1_1', 'SG_2_1_1', 'SG_3_1_1', 'SG_4_1_1', 'SG_5_1_1', 'G_plate_1_1_1', 'G_plate_2_1_1', 'G_plate_3_1_1', 'G_plate_4_1_1', 'G_plate_5_1_1', 'TS1_2cm', 'TS1_6cm', 'TS2_2cm', 'TS2_6cm', 'TS3_2cm', 'TS3_6cm','TS4_2cm', 'TS4_6cm', 'TS5_2cm', 'TS5_6cm', 'SWC_1_1_1', 'SWC_2_1_1', 'SWC_3_1_1', 'SWC_4_1_1', 'SWC_5_1_1', 'SW_IN', 'SW_OUT', 'LW_IN', 'LW_OUT', 'TA_1_1_3', 'T_CANOPY', 'CO2_sig_strgth_Min', 'H2O_sig_strgth_Min', 'CO2_density', 'H2O_density', 'LE', 'H', 'VPD', 'P', 'V_batt']


almonds = ["VAC", "OLA", "WWF", "WES"]
olives = ["ART_011", "ORO_022", "ORO_043", "COR_CS3"]
pistachios = ["BLS_001", "BLS_002"]
grapes = ["FLT", "SLC"]

In [20]:
# Read in parameters classified by equipment group
almond_list = pd.read_csv("C:/Users/mrcoo/Desktop/Audrey Work/Dashboard/Almond_Equipment.csv")
grape_list = pd.read_csv("C:/Users/mrcoo/Desktop/Audrey Work/Dashboard/Grape_Equipment.csv")
olive_list = pd.read_csv("C:/Users/mrcoo/Desktop/Audrey Work/Dashboard/Olive_Equipment.csv")
pistachio_list = pd.read_csv("C:/Users/mrcoo/Desktop/Audrey Work/Dashboard/Pistachio_Equipment.csv")

# Read in coordinates for sites
coords = pd.read_csv("C:/Users/mrcoo/Desktop/Audrey Work/Dashboard/Site_Long_Lat.csv")
area = pd.read_csv("C:/Users/mrcoo/Desktop/Audrey Work/Dashboard/Site_Area_Coords.csv")

img_path = "C:/Users/mrcoo/Desktop/Audrey Work/Dashboard/usda.png"

# Creates a dictionary that lists which parameters exist for each crop. Used to populate dropdowns based on crops later.

almond_dict = dict()
for j in almond_list.loc[0].unique():
    almond_dict[j] = []

for i in range(0, len(almond_list.loc[0])):
    almond_dict[almond_list.loc[0][i]].append(almond_list.columns[i])

almond_dict["All"] = sorted(list(almond_list.keys()), key = str.lower)
almond_dict["Reports"] = sorted(test_calls, key = str.lower)
    
grape_dict = dict()
for j in grape_list.loc[0].unique():
    grape_dict[j] = []

for i in range(0, len(grape_list.loc[0])):
    grape_dict[grape_list.loc[0][i]].append(grape_list.columns[i])

grape_dict["All"] = sorted(list(grape_list.keys()), key = str.lower)    
grape_dict["Reports"] = sorted(test_calls, key = str.lower)

olive_dict = dict()
for j in olive_list.loc[0].unique():
    olive_dict[j] = []

for i in range(0, len(olive_list.loc[0])):
    olive_dict[olive_list.loc[0][i]].append(olive_list.columns[i])
    
olive_dict["All"] = sorted(list(olive_list.keys()), key = str.lower)
olive_dict["Reports"] = sorted(reports, key = str.lower)
    
pistachio_dict = dict()
for j in pistachio_list.loc[0].unique():
    pistachio_dict[j] = []

for i in range(0, len(pistachio_list.loc[0])):
    pistachio_dict[pistachio_list.loc[0][i]].append(pistachio_list.columns[i])
    
pistachio_dict["All"] = sorted(list(pistachio_list.keys()), key = str.lower)
pistachio_dict["Reports"] = sorted(reports, key = str.lower)



last_week = relativedelta(weeks=+1)

# Assigns sites to sites.

site_dict = {
    "Almonds": ["OLA", "WWF", "VAC", "WES"],
    "Grapes": ["SLC", "FLT"],
    "Olives": ["ORO_022", "ORO_043", "COR_CS3", "ART_011"],
    "Pistachios": ["BLS_001", "BLS_002"],
    "Table Grapes": ["BRO_001"],
    "Custom": ["OLA", "WWF", "VAC", "WES", "SLC", "FLT", "ORO_022", 
               "ORO_043", "COR_CS3", "ART_011", "BLS_001", "BLS_002", "BRO_001"]
}
    

C:\Users\mrcoo\AppData\Local\Temp\ipykernel_18304\1531190915.py:20: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  almond_dict[almond_list.loc[0][i]].append(almond_list.columns[i])
C:\Users\mrcoo\AppData\Local\Temp\ipykernel_18304\1531190915.py:30: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  grape_dict[grape_list.loc[0][i]].append(grape_list.columns[i])
C:\Users\mrcoo\AppData\Local\Temp\ipykernel_18304\1531190915.py:40: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value 

In [21]:
# Dashboard

calls = test_calls

today = date.today()
ini = today - timedelta(days=6)
ini2 = today - timedelta(days=30)

app = Dash(__name__)

app.layout = [html.Div([
    html.Div([
        # Title and image at the top.
        
#         html.Div([
#             html.H1(["SAWS Data Dashboard"], 
#                 style = {"display": "inline-block", "margin-left": "37.5%", "vertical-align": "50%"}),
#             html.Img(src=app.get_asset_url('usda.png'), 
#                  style = {"width": 55, "display": "inline-block", "vertical-align": "10%", "margin-left": 10})
#         ]),
        
        
        # Container for crop radio selection and label, packaged with Divs strategically for styling
        
        html.Div([
            html.Div([
                html.Div([
                    html.Label(["Crops:"], style = {"text-align": "left", "font-weight": "bold", 
                                                "padding-left": 75})
                         ]),
                html.Div([
                    dcc.RadioItems(options = [
    #                 {"label": html.Div(["All"], style = {'display':'inline-block', "margin-right": 5, 
    #                                                       "padding-left": 2}), 
    #                  "value": "All"},
                    {"label": html.Div(["Almonds"], style = {'display':'inline-block', "margin-right": 5,
                                                             "padding-left": 2}), 
                     "value": "Almonds"},
                    {"label": html.Div(["Grapes"], style = {'display':'inline-block', "padding-left": 2,
                                                            "margin-right": 5}), 
                     "value": "Grapes"},
                    {"label": html.Div(["Olives"], style = {'display':'inline-block', "margin-right": 5,
                                                            "padding-left": 2}), 
                     "value": "Olives"},
                    {"label": html.Div(["Pistachios"], style = {'display':'inline-block', "margin-right": 5,
                                                                "padding-left": 2}),
                     "value": "Pistachios"},
                    {"label": html.Div(["Table Grapes"], style = {'display':'inline-block', "margin-right": 5,
                                                                "padding-left": 2}),
                     "value": "Table Grapes"},
                    {"label": html.Div(["Custom"], style = {'display':'inline-block', "margin-right": 5,
                                                                "padding-left": 2}),
                     "value": "Custom"}],
                value = "Almonds",
                id = "crop-radio",
                inline = True,
                style = {"margin-bottom": 10, "padding-left": 71, "margin-top": 10, "margin-bottom": 20,
                         "display": "inline-block"})
            ]),
        ]),
            
            
            # A couple Divs that contain the site selection drop down and its labels,
            # Second Site hidden on app load.
            
            html.Div([
                html.Label(["Sites:"], id = "site-label-1", 
                           style = {"text-align": "left", "font-weight": "bold", 
                                    "padding-left": 75})
            ]),
              
            html.Div([
                dcc.Dropdown(
                    id = 'site-selection',
                    multi = True,
                    searchable = False,
                    clearable = True,
                    maxHeight = 420,
                    style = {"margin-bottom": 10, "margin-top": 10, "margin-right": "15%"})
            ], style = {"width": "35%", "display": "inline-block", "padding-left": 75}),
            html.Div([
                html.Label(["Site 2 (For Parameter 2):"], id = "site-label-2",
                          style = {})
            ]),
            html.Div([
                dcc.Dropdown(
                    id = "second-site",
                    multi = False,
                    clearable = True,
                    searchable = False,
                    maxHeight = 420,
                    style = {})
            ], id = "second-site-div", style = {"width": "29.75%", "display": "inline-block", "padding-left": 75,
                                                "margin-top": 10}),





            # Div for equipment group selection dropdown
                
            html.Div([

                html.Div([
                    html.Label(["Equipment Group:"], style = {"text-align": "left", "font-weight": "bold",
                                                              "margin-top": 10})
            ], id = "equip-label", style = {"margin-top": 5}),
                dcc.Dropdown(
#                     options = ["Asperated Shield"],
#                     value = ["Asperated Shield"],
                    id = 'equip-group',
                    style = {"margin-bottom": 15, "margin-top": 10, "margin-right": "15%"},
                    clearable = False,
                    searchable = False,
                    maxHeight = 300)
                ], 
                style = {"width": "35%", "display": "inline-block",
                        "padding-left": 75}),
            
            # Dropdown for first parameter selection and label.
            
            html.Div([            
                html.Label(["Parameter:"],
                         id = "param-text-1",
                         style = {"text-align": "left", "font-weight": "bold", 
                        "padding-left": 75, 'display': "inline-block",
                        "padding-right": "24.5%"}),
                html.Label(["Plot Type:"], id = "plot-type-label", 
                           style = {"text-align": "center", "font-weight": "bold", 
                                    "margin-left": "12.8%", 'display': "inline-block"}),
            ]),

            html.Div([
                html.Div([
                    dcc.Dropdown(
        #                     options = ["e_probe"],
        #                     value = ["e_probe"],
                            id = "param-select",
                            style = {"margin-top": 10, "margin-right": "15%"},
                            clearable = False,
                            searchable = False,
                            maxHeight = 420)
                ], style = {"display": "inline-block", "width": "35%",
                            "padding-left": 75}),

                # Dropdown that customizes plot displayed based on used dropdown
                
                html.Div([
                    dcc.Dropdown(
                            options = [
                                {'label': "Line Plot: Parameter vs. Time", 'value': "LP-PT"},
                                {'label': "Line Plot: Two Parameters vs. Time", "value": "LP-2PT"},
                                {'label': "Scatter Plot: Parameter vs. Time", "value": "SP-PT"},
                                {'label': "Scatter Plot: Two Parameters vs. Time", "value": "SP-2PT"},
                                {'label': "Scatter Plot: Parameter vs. Parameter", 'value': "SP-PP"}],
                            value = "LP-PT",
                            multi = False,
                            clearable = False,
                            searchable = False,
                            id = 'plot-type-dropdown')
                ], style = {"display": "inline-block", "width": "35%", "padding-left": "7.5%"})
                
            ]),
            
            # Second parameter dropdown, hidden upon initialization
            
            html.Div([
                html.Label(["Parameter 2:"], id = "param-text-2",
                style = {"text-align": "left", "font-weight": "bold", 
                "padding-left": 75,
                "padding-right": "24.5%"})
                ], style = {"margin-top": 10}),
            html.Div([
                dcc.Dropdown(
                    id = "second-param",
                    style = {},
                    clearable = False,
                    searchable = False,
                    maxHeight = 420)
            ], style = {"width": "35%", "display": "inline-block", "padding-left": 75})

            
            
        ], style = {"display": "grid"}),
    
            
        ]),
        
        # Generates two graph objects: one for the map, one for the graph.
        
        html.Div([dcc.Graph(id = "map-graph",
                 style = {"width": "35%", "display": "inline-block", "padding-left": 75}),
#                  html.Div([""], style = {"display": "inline-block"}),
                 dcc.Graph(id = "norm-graph",
                 style = {"width": "57%", "display": "inline-flex", "margin-left": 35})]),
    
        # Creates storage for each non-site drop down to improve user experience.
    
        dcc.Store(id = "drop1-store",
                  data = {'drop1': 'option1'}),
        dcc.Store(id = "drop2-store",
                  data = {"drop2": "option2"}),
        dcc.Store(id = "drop3-store",
                  data = {"drop3": "option3"})],
        style = {"font-family": "Helvetica, sans-serif"})
             ]

# Fills equipment group dropdown based on radio selection. Alphabetizes equipment group names for continuity.

@callback(
    Output("equip-group", "options"),
    Input("crop-radio", "value"))

def populate_dropdown(radio):
    if radio == "Almonds":
        return [{'label': i, 'value': i} for i in sorted(almond_dict.keys())]
    elif radio == "Grapes":
        return [{'label': i, 'value': i} for i in sorted(grape_dict.keys())]
    elif radio == "Olives":
        return [{'label': i, 'value': i} for i in sorted(olive_dict.keys())]
    elif radio == "Pistachios":
        return [{'label': i, 'value': i} for i in sorted(pistachio_dict.keys())]
    elif radio == "Table Grapes":
        return [{'label': i, 'value': i} for i in sorted(olive_dict.keys())]
    else:
        return [{'label': i, 'value': i} for i in sorted(olive_dict.keys())]


# Populates second dropdown based on equipment selected in first dropdown.

@callback(
    Output("param-select", "options"),
    Input("equip-group", "value"),
    Input("crop-radio", "value"))

def talking_dropdown(selected_param, radio):
    if radio == "Almonds":
        return [{'label': i, 'value': i} for i in almond_dict[selected_param]]
    elif radio == "Grapes":
        return [{'label': i, 'value': i} for i in grape_dict[selected_param]]
    elif radio == "Olives":
        return [{'label': i, 'value': i} for i in olive_dict[selected_param]]
    elif radio == "Pistachios":
        return [{'label': i, 'value': i} for i in pistachio_dict[selected_param]]
    elif radio == "Table Grapes":
        return [{'label': i, 'value': i} for i in olive_dict[selected_param]] 
    else:
        return [{'label': i, 'value': i} for i in olive_dict[selected_param]]
    
    
# Repeat same code as above but for second parameter dropdown.
    
@callback(
    Output("second-param", "options"),
    Input("equip-group", "value"),
    Input("crop-radio", "value"))

def second_talking_dropdown(selected_param, radio):
    if radio == "Almonds":
        return [{'label': i, 'value': i} for i in almond_dict[selected_param]]
    elif radio == "Grapes":
        return [{'label': i, 'value': i} for i in grape_dict[selected_param]]
    elif radio == "Olives":
        return [{'label': i, 'value': i} for i in olive_dict[selected_param]]
    elif radio == "Pistachios":
        return [{'label': i, 'value': i} for i in pistachio_dict[selected_param]]
    elif radio == "Table Grapes":
        return [{'label': i, 'value': i} for i in olive_dict[selected_param]] 
    else:
        return [{'label': i, 'value': i} for i in olive_dict[selected_param]]

# Sets site options based on crop selection, exception granted for "custom" option.
    
@callback(
    Output("site-selection", "options"),
    Input("crop-radio", "value"))
def site_dropdown(radio):
        if radio != "Custom":
            return [{'label': i, 'value': i} for i in site_dict[radio]]
        else:
            return [{'label': i, 'value': i} for i in site_dict["Custom"]]

# Stores current equipment value selected
    
@callback(
    Output('drop1-store', 'data'),
    Input('crop-radio', 'value'),
    State('equip-group', 'value'), 
)

def update_store1(radio_value, drop1_value):
    return {'drop1': drop1_value}

# Stores current parameter selected

@callback(
    Output('drop2-store', 'data'),
    Input('crop-radio', 'value'),
    State('param-select', 'value'), 
)

def update_store2(radio_value, drop2_value):
    return {'drop2': drop2_value}

# Stores the second parameter.

@callback(
    Output("drop3-store", "data"),
    Input("crop-radio", "value"),
    State("second-param", "value"))

def update_store3(radio_value, drop3_value):
    return {'drop3': drop3_value}


# Maintains equipment when changing radio items
# Initializes equipment group and prevents invalid equipment group for being selected for certain crops

@callback(
    Output('equip-group', 'value'),
    Input('drop1-store', 'data'),
    Input("equip-group", "options"),
    Input('crop-radio', 'value')
)
def update_dropdowns1(store_data, options, crop):
    if crop != "Almonds" and (store_data['drop1'] == "IRT Sensor" or store_data['drop1'] == "Fine Wire Thermocouple"):
        return "Asperated Shield"
    elif store_data['drop1'] == None:
        return options[0]['value']
    else:
        return store_data['drop1']

# Maintains parameter when changing radio items
# If no equipment has been selected, default will populate menu     
    
@callback(
    Output('param-select', 'value'),
    Input('drop2-store', 'data'),
    Input('param-select', 'options'),
    Input('crop-radio', 'value'),
    Input('equip-group', 'value')
)
def update_dropdowns2(store_data, options, site, equipment):
    if store_data["drop2"] == None:
        return options[0]['value']
    elif site == "Almonds":
        if store_data['drop2'] in almond_dict[equipment]:
            return store_data['drop2']
        else:
            return options[0]['value']
    elif site == "Olives":
        
        # Checks to see if current selected parameter exists for site, 
        # if not displays first parameter under equipment
        
        if store_data['drop2'] in olive_dict[equipment]:
            return store_data['drop2']
        else:
            return options[0]['value']
    elif site == "Pistachios":
        if store_data['drop2'] in pistachio_dict[equipment]:
            return store_data['drop2']
        else:
            return options[0]['value']
    elif site == "Grapes":
        if store_data['drop2'] in grape_dict[equipment]:
            return store_data['drop2']
        else:
            return options[0]['value']
    elif site == "Table Grapes":
        if store_data["drop2"] in olive_dict[equipment]:
            return store_data['drop2']
        else:
            return options[0]['value']
    else:
        if store_data["drop2"] in olive_dict[equipment]:
            return store_data['drop2']
        else:
            return options[0]['value']
        

# Almost identical code as above, used for secondary parameter dropdown.
        
@callback(
    Output("second-param", 'value'),
    Input('drop3-store', 'data'),
    Input('second-param', 'options'),
    Input('crop-radio', 'value'),
    Input('equip-group', 'value')
)

def update_second_parameter(store_data, options, site, equipment):
    if store_data["drop3"] == None:
        return options[1]['value']
    elif site == "Olives":
        
        # Checks to see if current selected parameter exists for site, 
        # if not displays first parameter under equipment
        
        if store_data['drop3'] in olive_dict[equipment]:
            return store_data['drop3']
        else:
            return options[1]['value']
    elif site == "Almonds":
        if store_data['drop3'] in almond_dict[equipment]:
            return store_data['drop3']
        else:
            return options[1]['value']
    elif site == "Pistachios":
        if store_data['drop3'] in pistachio_dict[equipment]:
            return store_data['drop3']
        else:
            return options[1]['value']
    elif site == "Grapes":
        if store_data['drop3'] in grape_dict[equipment]:
            return store_data['drop3']
        else:
            return options[1]['value']
    elif site == "Table Grapes":
        if store_data["drop3"] in olive_dict[equipment]:
            return store_data['drop3']
        else:
            return options[1]['value']
    else:
        if store_data["drop3"] in olive_dict[equipment]:
            return store_data['drop3']
        else:
            return options[1]['value']
        
# Populate dropdown depending on crop selection and number of parameters viewing

@callback(
    Output("site-selection", "value"),
    Input("site-selection", "options"),
    Input("crop-radio", "value"),
    Input("plot-type-dropdown", "value"))

def site_populate(options, radio, num_param):
    if num_param in ['LP-PT', 'SP-PT']:
        if radio == "Custom":
            return site_dict["Custom"]
        else:
            return [options[i]["value"] for i in range(0,len(site_dict[radio]))]
    else:
        return options[0]["value"]

# Switch on/off second dropdown based on # of parameters
    
@callback(
    Output("second-param", "style"),
    Input("plot-type-dropdown", "value"))

def add_second_dropdown(num_param):
    if num_param in ['LP-PT', 'SP-PT']:
        return {"display": "none"}
    else:
        return {"margin-top": 5, "margin-right": "15%", "margin-bottom": 10}
    
# Disables/Enabled ability to selected multiple sites depending on plot type.

@callback(
    Output("site-selection", "multi"),
    Input("plot-type-dropdown", "value"))

def adjust_site_dropdown(num_param):
    if num_param in ['LP-PT', 'SP-PT']:
        return True
    else:
        return False
    
# Creates options for the second site, depending on what crop is selected.
    
@callback(
    Output("second-site", "options"),
    Input("crop-radio", "value"))

def second_site_options(radio):
    if radio != "Custom":
            return [{'label': i, 'value': i} for i in site_dict[radio]]
    else:
        return [{'label': i, 'value': i} for i in site_dict["Custom"]]
    
# Initializes a site for second site dropdown

@callback(
    Output("second-site", "value"),
    Input("second-site", "options"))

def second_site_drop_intialize(options):
    return options[0]["value"]

# Hides/shows second site style based on inputted plot type dropdown

@callback(
    Output("second-site", "style"),
    Input("plot-type-dropdown", "value"))

def show_second_site(num_param):
    if num_param in ['LP-PT', 'SP-PT']:
        return {"display": "none"}
    else:
        {"margin-top": 5, "margin-right": "15%", "margin-bottom": 10}
        
# Adjusts placement of elemtents based on plot type dropdown input.
        
@callback(
    Output("equip-label", "style"),
    Output("plot-type-label", "style"),
    Input("plot-type-dropdown", "value"))

def change_equip_label(plot_type):
    if plot_type in ["LP-PT", "SP-PT"]:
        return [{}, {"text-align": "center", "font-weight": "bold", 
                                    "margin-left": "12.8%", 'display': "inline-block"}]
    else:
        return [{"margin-top": 10},
                {"text-align": "center", "font-weight": "bold", "margin-left": "11.9%", 'display': "inline-block"}]

# Edits/hides/shows site dropdown based on plot type dropdown
    
@callback(
    Output("site-label-1", "children"),
    Output("site-label-2", "style"),
    Input("plot-type-dropdown", "value"))

def site_text_update(plot_type):
    if plot_type in ['LP-PT', 'SP-PT']:
        return ["Sites:", {"display": "none"}]
    else:
        return ["Site 1 (For Parameter 1):",
               {"text-align": "left", "font-weight": "bold", "padding-left": 75}]

# Hides/Shows/Edits labels of parameter dropdown labels based on plot type selection

@callback(
    Output("param-text-1", "children"),
    Output("param-text-2", "style"),
    Input("plot-type-dropdown", "value"))

def param_text_update(plot_type):
    if plot_type in ["LP-PT", "SP-PT"]:
        return ["Parameter:", {"display": "none"}]
    else:
        return ["Parameter 1:", 
               {"text-align": "left", "font-weight": "bold", "padding-left": 75, "padding-right": "24.5%"}]

# Generates and customizes map.
# Displays sites on map depending on user input from radio items.

@callback(
    Output("map-graph", "figure"),
    Input("crop-radio", "value"),
    Input("site-selection", "value"))

def plot_map(sites, custom_sites):
    
    # Sets default zoom for plot depending on site, some crops have sites that are very close to/far from each other.
    
    if sites != "Custom":
        coords_temp = coords[coords["Crop"] == sites]
        area_temp = area[area["Crop"] == sites]
        if sites != "Almonds":
            h_set = 8
        else:
            h_set = 6.1
    else:
        try:
            coords_temp = coords[coords["Site"].isin(custom_sites)]
            h_set = 5.75

        except:
            coords_temp = coords[coords["Site"] == custom_sites]
            h_set = 8

        
    fig = px.scatter_mapbox(coords_temp, lat = "Lat", lon = "Lon", hover_name = "Site",
                            zoom = h_set, height = 450,
                            center = {"lat": mean(coords_temp.iloc[:,0]), "lon": mean(coords_temp.iloc[:,1])},
                            color = "Crop",
                            color_discrete_map={
                            "Almonds": "Blue",
                            "Olives": "Red",
                            "Pistachios": "Green",
                            "Grapes": "Purple",
                            "Table Grapes": "Yellow"}
                            )
    
#     # Creates boundaries for each ranch.
    
#     for ranch in range(0, len(area_temp)):
#         fig.add_trace(go.Scattermapbox(
#             mode = "lines",
#             hoverinfo = "none",
#             showlegend = False,
#             lon = area_temp.iloc[ranch,0:5],
#             lat = area_temp.iloc[ranch,5:10],
#             marker = {'size': 8},
#             line = go.scattermapbox.Line(color = area_temp.iloc[ranch,12], width = 2)
#         ))
        
#     # Makes site markers as top layer, fixes buggy hover feature.
#     fig.data = tuple(fig.data[::-1])
    # Chooses map background type
    fig.update_layout(mapbox_style="open-street-map")
    fig.update_traces({'marker':{'size': 8}})
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":75})
    # Puts legend in top left corner of map.
    fig.update_layout(legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.01
))
    return fig

# Generates a plot based on selected parameter and crop.

@callback(
    Output("norm-graph", "figure"),
    Input("crop-radio", "value"),
    Input("param-select", "value"),
    Input("plot-type-dropdown", "value"),
    Input("site-selection", "value"),
    Input("second-param", "value"),
    Input("second-site", "value"))

def plot_graph(crops, yaxis_column_name, plot, first_drop, second_param, second_site):
    if plot in ['LP-PT', 'SP-PT']:
        site_drop = first_drop
    else:
        site_drop = [first_drop, second_site]
    # Generates a data frame for plot based on crop/site selected.
    if crops == "Almonds" or crops == "Grapes":
        sites = site_drop
        try:
            temp = trex_all.Site.isin(sites)
            data_temp = trex_all[temp]
        except:
            data_temp = trex_all[trex_all["Site"] == sites]
    elif crops == "Olives" or crops == "Pistachios" or crops == "Table Grapes":
        sites = site_drop
        try:
            temp = matt_all.Site.isin(sites)
            data_temp = matt_all[temp]
        except:
            data_temp = matt_all[matt_all["Site"] == sites]
    else:
        sites = site_drop
        combine_df = pd.concat([trex_all,matt_all])
        combine_df["V_batt"] = combine_df["V_batt"].combine_first(combine_df["batt_volt"])
        print(type(sites))
        try:
            temp = combine_df.Site.isin(sites)
            data_temp = combine_df[temp]
        except:
            data_temp = combine_df[combine_df["Site"] == sites]
            
        
    # Dash will use generate one plot depending on user input, data is the same, plots have slight visual tweaks. 
    
    if plot == "LP-PT" or plot == "SP-PT":
        if plot == "LP-PT":
            fig = px.line(data_temp, x = data_temp.TIMESTAMP, y = data_temp[yaxis_column_name], custom_data = ["Site"],
                         color = data_temp.Site,
                         color_discrete_map={
                        "BLS_001": "lime",
                        "BLS_002": "blue",
                        "ORO_022": "goldenrod",
                        "ORO_043": "green",
                        "COR_CS3": "magenta",
                        "ART_011": "red",
                        "OLA": "olive",
                        "WWF": "steelblue",
                        "VAC": "saddlebrown",
                        "SLC": "orange",
                        "FLT": "black",
                        "WES": "darkmagenta",
                        "BRO_001": "gold"},
                         height = 490)
            fig.update_traces(line={'width': 1.5})

        else:
            fig = px.scatter(data_temp, x = data_temp.TIMESTAMP, y = data_temp[yaxis_column_name], color = data_temp.Site,
                         custom_data = ["Site"],
                         color_discrete_map={
                        "BLS_001": "lime",
                        "BLS_002": "blue",
                        "ORO_022": "goldenrod",
                        "ORO_043": "green",
                        "COR_CS3": "magenta",
                        "ART_011": "red",
                        "OLA": "olive",
                        "WWF": "steelblue",
                        "VAC": "saddlebrown",
                        "SLC": "orange",
                        "FLT": "black",
                        "WES": "darkmagenta",
                        "BRO_001": "gold"},
                         height = 490)
        # Assigns starting range for plot based on parameter.
        # Still allows user to pan up/down and left/right.

        ylow = int(rangedf[yaxis_column_name][0])
        yhi = int(rangedf[yaxis_column_name][1])
        fig.update_yaxes(range = [ylow, yhi], fixedrange = False)
        
        # Customized box on hover which shows date, time, y-value, and site.
    
        fig.update_traces(hovertemplate = '<b>%{customdata}</b><br>' +
                  'TIMESTAMP: %{x|%Y-%m-%d %H:%M}<br>'+yaxis_column_name+': %{y}<extra></extra>')
        fig.update_layout(legend_title_text = "Sites (Click to Toggle)")
        fig.update_layout(
        xaxis=dict(
        rangeselector=dict(
            buttons=list([
                dict(count=1,
                     label="1D",
                     step="day",
                     stepmode="backward"),
                dict(count=7,
                     label="1W",
                     step="day",
                     stepmode="backward"),
                dict(count=1,
                     label="1M",
                     step="month",
                     stepmode="backward"),
                dict(step="all")
            ])
        ),
        rangeslider=dict(
            visible=True,
            thickness = .1,
        ),
        type="date"
    )
)
    else:
        if plot != "SP-PP":
            # Create a new data frame to allow plotting command to use "group" option for organization, color ease.
            data = {
                "date": data_temp.TIMESTAMP,
                yaxis_column_name: data_temp[yaxis_column_name],
                second_param: data_temp[second_param],
                "Site": data_temp.Site
            }
            df = pd.DataFrame(data)
            print(df)
            df_long = df.melt(id_vars=['date', "Site"], 
                              value_vars=[yaxis_column_name, second_param],
                              var_name='param_type',
                              value_name='param_vals')
            df_long_site_1 = df_long[df_long["Site"] == site_drop[0]]
            df_long_site_1 = df_long_site_1[df_long_site_1["param_type"] == yaxis_column_name]
            df_long_site_2 = df_long[df_long["Site"] == site_drop[1]]
            df_long_site_2 = df_long_site_2[df_long_site_2["param_type"] == second_param]

            if plot == "LP-2PT":
                fig = px.line(df_long_site_1, x = df_long_site_1.date, 
                             y = df_long_site_1.param_vals,
                             color = df_long_site_1.param_type,
                             custom_data = ["param_type"],
                             height = 490)
                fig.add_trace(
                    go.Scatter(
                        x = df_long_site_2.date,
                        y = df_long_site_2.param_vals,
                        mode = "lines",
                        name = second_param
                    ))
                fig.update_traces(line={'width': 1.5})
                
                range_options = [int(rangedf[yaxis_column_name][0]), int(rangedf[yaxis_column_name][1]),
                     int(rangedf[second_param][0]), int(rangedf[second_param][1])]
                ylow = min(range_options)
                yhi = max(range_options)
                fig.update_yaxes(range = [ylow, yhi], fixedrange = False)
                fig.update_traces(
                hovertemplate = '<b>TIMESTAMP</b>: %{x|%Y-%m-%d %H:%M}<br>' +
                              '<b>Y-Value</b>: %{y}<br>')

            else:
                fig = px.scatter(df_long_site_1, x = df_long_site_1.date, 
                             y = df_long_site_1.param_vals,
                             color = df_long_site_1.param_type,
                             custom_data = ["param_type"],
                             height = 490)
                fig.add_trace(
                    go.Scatter(
                        x = df_long_site_2.date,
                        y = df_long_site_2.param_vals,
                        name = second_param,
                        mode = "markers"
                    ))
                fig.update_traces(
                hovertemplate = '<b>TIMESTAMP</b>: %{x|%Y-%m-%d %H:%M}<br>' +
                              '<b>Y-Value</b>: %{y}<br>')
                range_options = [int(rangedf[yaxis_column_name][0]), int(rangedf[yaxis_column_name][1]),
                     int(rangedf[second_param][0]), int(rangedf[second_param][1])]
                ylow = min(range_options)
                yhi = max(range_options)
                fig.update_yaxes(range = [ylow, yhi], fixedrange = False)
            fig.update_layout(
            xaxis=dict(
            rangeselector=dict(
                buttons=list([
                    dict(count=1,
                         label="1D",
                         step="day",
                         stepmode="backward"),
                    dict(count=7,
                         label="1W",
                         step="day",
                         stepmode="backward"),
                    dict(count=1,
                         label="1M",
                         step="month",
                         stepmode="backward"),
                    dict(step="all")
                ])
            ),
            rangeslider=dict(
                visible=False,
                thickness = .1,
            ),
            type="date"
            ))
        else:
            if site_drop[0] == site_drop[1]:
                pvp_df = data_temp[[yaxis_column_name, second_param]]
                fig = px.scatter(pvp_df, x = pvp_df.iloc[:,0],
                             y = pvp_df.iloc[:,1],
                            height = 490)
            else:
                site_1 = data_temp[data_temp["Site"] == site_drop[0]]
                site_1 = site_1[["TIMESTAMP", yaxis_column_name]]
                site_2 = data_temp[data_temp["Site"] == site_drop[1]]
                site_2 = site_2[["TIMESTAMP", second_param]]
                plot_ready = pd.merge(site_1, site_2, on='TIMESTAMP')
                fig = px.scatter(plot_ready, x = plot_ready.iloc[:,1],
                                 y = plot_ready.iloc[:,2],
                                height = 490)

            xlow = int(rangedf[yaxis_column_name][0])
            xhi = int(rangedf[yaxis_column_name][1])
            ylow = int(rangedf[second_param][0])
            yhi = int(rangedf[second_param][1])
            fig.update_yaxes(range = [ylow, yhi], fixedrange = False) 
            fig.update_xaxes(range = [xlow, xhi], fixedrange = False)
            fig.update_layout(plot_bgcolor= "#efefef")

                # Sets default plot setting as pan instead of zoom.
            fig.update_layout(dragmode='pan')
            fig.update_layout(xaxis_title = yaxis_column_name + " from " + site_drop[0],
                              yaxis_title = second_param + " from " + site_drop[1])
            fig.update_traces(
                hovertemplate = '<b>'+yaxis_column_name+' ('+site_drop[0]+')</b>: %{x}<br>' +
                              '<b>'+second_param+' ('+site_drop[1]+')</b>: %{y}<br>'
            )
            fig.update_layout(title = "Data for Sites: " + site_drop[0] + " and " + site_drop[1])
            return fig
        
        # Set a unique range based on the two normal ranges of selected parameters.
        

        
        # Customized box on hover which shows date, time and parameter.
        # Tweaked from command under "if" statement to emphasize parameter and not site.
    
#         fig.update_traces(hovertemplate = '<b>'+site_drop+'</b><br>'
#                           'TIMESTAMP: %{x|%Y-%m-%d %H:%M}<br>'+
#                           '%{customdata}'+': %{y}<extra></extra>')
        fig.update_layout(legend_title_text = "Parameters (Click to Toggle)",
                          yaxis_title = site_drop[0] + ': ' + yaxis_column_name +
                          " / "+ site_drop[1] + ': ' + second_param,
                          xaxis_title = "TIMESTAMP",
                          title = "Data for Sites: " + site_drop[0] + " and " + site_drop[1])

        
    fig.update_traces({'marker':{'size':3.5}})

    
    # Changes date display on tick marks
    
    fig.update_xaxes(tickformat = "%m/%d\n%Y")
    
    # Sets default range from one week ago to today.
    
    fig.update_xaxes(range = [max(data_temp.TIMESTAMP) - last_week, max(data_temp.TIMESTAMP)])
    
    fig.update_layout(plot_bgcolor= "#efefef")
    
    # Sets default plot setting as pan instead of zoom.
    fig.update_layout(dragmode='pan')
    
    # Creates a slider to view different x-axis range easily. 
    # Can manually change range with slider or use buttons that show data from last day, week, month.
    # All options are negotiable just placeholder for now.
    
    
    return fig

if __name__ == '__main__':
    app.run(debug=True)
#     app.run_server(host="0.0.0.0", port="8050")
#     Will be put on wifi if launched

<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
<class 'list'>
                     date  CO2_density Site
21670 2024-12-16 08:30:00     895.0464  SLC
21671 2024-12-16 09:00:00     878.2444  SLC
21672 2024-12-16 09:30:00     888.0608  SLC
21673 2024-12-16 10:00:00     871.0668  SLC
21674 2024-12-16 10:30:00     860.4304  SLC
...                   ...          ...  ...
25546 2025-03-18 06:00:00     852.5851  SLC
25547 2025-03-18 06:30:00     864.8768  SLC
25548 2025-03-18 07:00:00     879.8066  SLC
25549 2025-03-18 07:30:00     850.4519  SLC
25550 2025-03-18 08:00:00     818.8060  SLC

[3881 rows x 3 columns]
                     date  CO2_density  CO2_sig_strgth_Min Site
21670 2024-12-16 08:30:00     895.0464            0.434838  SLC
21671 2024-12-16 09:00:00     878.2444            0.331697  SLC
21672 2024-12-16 09:30:00     888.0608            0.320469  SLC
21673 2024-12-16 10:00:00     871.0668   

In [ ]:
df1 = pd.DataFrame({
    'time': ['2024-08-01', '2024-08-02', '2024-08-03'],
    'value1': [10, 20, 30]
})

df2 = pd.DataFrame({
    'time': ['2024-08-01', '2024-08-02', '2024-08-03'],
    'value2': [100, 200, 300]
})

# Merge the data frames on the 'time' column
merged_df = pd.merge(df1, df2, on='time')

# Display the merged data frame
print(merged_df)

In [ ]:
matt_all[["Site", "Site"]]